# Client

Client code interacting with a server that implements: 

* custom streaming for an agent
* agent with user selected tools

This agent does not have memory! See other examples in LangServe to see how to add memory.

**ATTENTION** We made the agent stream strings as an output. This is almost certainly not what you would want for your application. Feel free to adapt to return more structured output; however, keep in mind that likely the client can just use `astream_events`!

See relevant documentation about agents:

* Creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
* Streaming with agents: https://python.langchain.com/docs/modules/agents/how_to/streaming#custom-streaming-with-events
* General streaming documentation: https://python.langchain.com/docs/expression_language/streaming

You can interact with this via API directly

In [1]:
word = "audioeeeeeeeeeee"
len(word)

16

In [2]:
import requests

inputs = {"input": {"input": f"what is the length of the word {word}?", "chat_history": [], "tools": []}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

print(response.json()['output'])


Starting agent: agent with input: {'input': 'what is the length of the word audioeeeeeeeeeee?'}
The length of the word "audioeeeeeeeeeee" is 15 characters.
Done agent: agent with output: The length of the word "audioeeeeeeeeeee" is 15 characters.



Let's provide it with a tool to test that tool selection works

In [3]:
import requests

inputs = {"input": {"input": f"what is the length of the word {word}?", "chat_history": [], "tools": ["word_length", "favorite_animal"]}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

print(response.json()['output'])


Starting agent: agent with input: {'input': 'what is the length of the word audioeeeeeeeeeee?'}

Starting tool: word_length with inputs: {'word': 'audioeeeeeeeeeee'}

Done tool: word_length with output: 16
The length of the word "audioeeeeeeeeeee" is 16.
Done agent: agent with output: The length of the word "audioeeeeeeeeeee" is 16.



You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [4]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

## Stream

Streaming output from a **CUSTOM STREAMING** implementation that streams string representations of intermediate steps. Please see server side implementation for details.

In [7]:
async for chunk in remote_runnable.astream({"input": "What is eugenes favorite animal?", "tools": ["word_length"]}):
    print(chunk, end='|', flush=True)


|Starting agent: agent with input: {'input': 'What is eugenes favorite animal?'}|
|I|'m| sorry|,| but| I| don|'t| have| access| to| personal| information| about| individuals| unless| it| has| been| shared| with| me| in| the| course| of| our| conversation|.|
|Done agent: agent with output: I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation.|
|

In [8]:
async for chunk in remote_runnable.astream({"input": "What is eugenes favorite animal?", "tools": ["word_length", "favorite_animal"]}):
    print(chunk, end='|', flush=True)


|Starting agent: agent with input: {'input': 'What is eugenes favorite animal?'}|
|
|Starting tool: favorite_animal with inputs: {'name': 'Eugene'}|
|
|Done tool: favorite_animal with output: cat|
|E|ug|ene|'s| favorite| animal| is| a| cat|.|
|Done agent: agent with output: Eugene's favorite animal is a cat.|
|